In [175]:
import pandas as pd  # Data manipulation and analysis
import numpy as np  # Numerical operations and array handling
import matplotlib.pyplot as plt  # Plotting and visualization
import seaborn as sns  # Statistical data visualization
from sklearn import datasets, model_selection, preprocessing, metrics  # Machine learning tools
import tensorflow
import tensorflow as tf  # Deep learning framework
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime
import torch  # Deep learning framework
import torchvision  # Computer vision tools for PyTorch
import plotly.express as px  # Interactive plotting
import plotly.graph_objects as go  # More interactive plotting options
import bokeh.plotting as bkp  # Interactive visualization library
import altair as alt  # Declarative statistical visualization library
import gensim  # Topic modeling and document similarity
from textblob import TextBlob  # Text processing and NLP
import pyLDAvis  # Visualization of topic models
import dask.dataframe as dd  # Parallel computing with pandas-like DataFrames
import pyarrow as pa  # Columnar data format for efficient data interchange
import fastparquet  # Parquet file format for efficient data storage
import cv2  # Computer vision and image processing
from skimage import io, filters, color  # Image processing
import albumentations as A  # Image augmentation library
import gym  # Toolkit for developing and comparing reinforcement learning algorithms
from stable_baselines3 import PPO  # Reinforcement learning algorithms
from tqdm import tqdm  # Progress bar for loops
import joblib  # Serialization and deserialization of Python objects
from pydantic import BaseModel  # Data validation and settings management using Python type annotations
from transformers import pipeline  # State-of-the-art Natural Language Processing
import spacy  # Industrial-strength NLP
import nltk  # Natural Language Toolkit
import re
import openai  # OpenAI API for accessing GPT models
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler,LabelEncoder
import pickle

In [176]:
data=pd.read_csv("Churn_Modelling.csv")
data=data.drop(['RowNumber','CustomerId','Surname'],axis=1)
label_encoder_gender=LabelEncoder()
data['Gender']=label_encoder_gender.fit_transform(data['Gender'])

In [177]:
onehot_encoder_geo=OneHotEncoder()
geo_encoder=onehot_encoder_geo.fit_transform(data[['Geography']]).toarray()
onehot_encoder_geo.get_feature_names_out(['Geography'])

geo_encoded_df=pd.DataFrame(geo_encoder,columns=onehot_encoder_geo.get_feature_names_out(['Geography']))

In [178]:
data=pd.concat([data.drop('Geography',axis=1),geo_encoded_df],axis=1)
data

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,771,1,39,5,0.00,2,1,0,96270.64,0,1.0,0.0,0.0
9996,516,1,35,10,57369.61,1,1,1,101699.77,0,1.0,0.0,0.0
9997,709,0,36,7,0.00,1,0,1,42085.58,1,1.0,0.0,0.0
9998,772,1,42,3,75075.31,2,1,0,92888.52,1,0.0,1.0,0.0


In [179]:
## Save the encoders and sscaler
with open('label_encoder_gender.pkl','wb') as file:
    pickle.dump(label_encoder_gender,file)

with open('onehot_encoder_geo.pkl','wb') as file:
    pickle.dump(onehot_encoder_geo,file)

In [180]:
## DiVide the dataset into indepent and dependent features
X=data.drop('Exited',axis=1)
y=data['Exited']

## Split the data in training and tetsing sets
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

## Scale these features
scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)

In [181]:
with open('standard_scaler.pkl','wb') as file:
    pickle.dump(scaler,file) ## Save standard scaler in pickle format.

ANN 

In [182]:
## Build Our ANN Model

# 1. Initialize sequential model.
model=Sequential([
    Dense(64,activation='relu',input_shape=(X_train.shape[1],)), ## HL1 Connected wwith input layer , X_train.shape is used to identify the total column size & can be used as my input. Here there are 12 columns so input = 12. These inputs get connected to my hidden layers.  
    Dense(32,activation='relu'), ## HL2
    Dense(1,activation='sigmoid')  ## output layer
]

)
model.summary()

c:\Program Files\Python310\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_9"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_27 (Dense)                │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_28 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_29 (Dense)                │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

In [183]:
opt=tensorflow.keras.optimizers.Adam(learning_rate=0.01) # Adam optimizer has a fixed learning rate but we can give our own.
loss=tensorflow.keras.losses.BinaryCrossentropy() # We can set our loss which needs to be reduced. 

## compile the model
model.compile(optimizer=opt,loss="binary_crossentropy",metrics=['accuracy'])

## Set up the Tensorboard

log_dir="logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S") # Directory or folder creation. 
tensorflow_callback=TensorBoard(log_dir=log_dir,histogram_freq=1) # Tensorboard visuaizes all of our logs during model training & histogram development happens.

## Set up Early Stopping for the epochs if you see loss value is not decreasing.
early_stopping_callback=EarlyStopping(monitor='val_loss',patience=100,restore_best_weights=True) # We monitor the model to take early stopping decision if loss is not decreased but be patient till 10 epochs. At whatever epoch you find best weights just restore it & stop early stopping.

In [184]:
### Train the model
history=model.fit( X_train,y_train,validation_data=(X_test,y_test),epochs=500,callbacks=[tensorflow_callback,early_stopping_callback])

# In each epoch we can see accuracy of main data , accuracy of validating data , epoch will be patient till 100th run before giving any big result.

Epoch 1/500


250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7954 - loss: 0.4631 - val_accuracy: 0.8520 - val_loss: 0.3657
Epoch 2/500
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8578 - loss: 0.3498 - val_accuracy: 0.8575 - val_loss: 0.3441
Epoch 3/500
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8620 - loss: 0.3435 - val_accuracy: 0.8595 - val_loss: 0.3408
Epoch 4/500
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8591 - loss: 0.3357 - val_accuracy: 0.8605 - val_loss: 0.3444
Epoch 5/500
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8602 - loss: 0.3457 - val_accuracy: 0.8490 - val_loss: 0.3536
Epoch 6/500
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8613 - loss: 0.3342 - val_accuracy: 0.8570 - val_loss: 0.3557
Epoch 7/500
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8635 - loss: 0.3344 - val_accuracy: 0.8565 - val_loss: 0.3436
Epoch 8/500
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8590 - loss: 0.3349 - val_accuracy: 0.8625

In [185]:
model.save('model.h5')

In [186]:
## Load Tensorboard Extension
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [187]:
%tensorboard --logdir logs/fit # This has all of our model information from scratch.

Reusing TensorBoard on port 6006 (pid 13784), started 1 day, 7:33:10 ago. (Use '!kill 13784' to kill it.)

In [193]:
from tensorflow.keras.models import load_model

### Load the trained model, scaler pickle,onehot

model=load_model('model.h5')

## load the encoder and scaler
with open('onehot_encoder_geo.pkl','wb') as file:
    pickle.dump(onehot_encoder_geo,file)

with open('label_encoder_gender.pkl', 'rb') as file:
    label_encoder_gender = pickle.load(file)

with open('standard_scaler.pkl','wb') as file:
    pickle.dump(scaler,file) 

In [ ]:
# Example input data to check at first pass. (Key value pairs) !!

input_data = {
    'CreditScore': 600,
    'Geography': 'France',
    'Gender': 'Male',
    'Age': 40,
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000
}



In [196]:
# One-hot encode column 'Geography'
geo_encoded = onehot_encoder_geo.transform([[input_data['Geography']]]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=onehot_encoder_geo.get_feature_names_out(['Geography']))

# Create input DataFrame
input_df = pd.DataFrame([input_data])

# Encode categorical variables
# Label encoding Gender
input_df['Gender'] = label_encoder_gender.transform(input_df['Gender'])

# Concatenate the original DataFrame (excluding 'Geography') with the one-hot encoded DataFrame
input_df = pd.concat([input_df.drop("Geography", axis=1), geo_encoded_df], axis=1)

# Display the final DataFrame
input_df

c:\Program Files\Python310\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [200]:
input_scaled = scaler.transform(input_df)
prediction = model.predict(input_scaled) ## The predict method generates predictions based on the scaled input data. The output is typically an array of probabilities or class labels.
prediction_proba = prediction[0][0]
prediction_proba 

## Purpose: This line extracts the probability of the customer churning from the prediction array.
## Explanation: Assuming the model's output is a 2D array, prediction[0][0] accesses the first element, which represents the probability of churn.

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step


0.019982623

In [201]:
if prediction_proba > 0.5:
    print('The customer is likely to churn.')
else:
    print('The customer is not likely to churn.')

## If the probability of churn (prediction_proba) is greater than 0.5, it prints that the customer is likely to churn. Otherwise, it prints that the customer is not likely to churn.

The customer is not likely to churn.
